In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)  # AG News has 4 categories

# Load AG News dataset
ag_news = load_dataset("ag_news")

# Tokenize
def preprocess(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

encoded = ag_news.map(preprocess, batched=True)
encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Setup Trainer
training_args = TrainingArguments(
    output_dir="./ag_news_pretrained",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded['train'],
    eval_dataset=encoded['test'],
)

trainer.train()

/opt/anaconda3/envs/llm_coursework/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/envs/llm_coursework/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Load your pretrained AG News model
model = BertForSequenceClassification.from_pretrained("./ag_news_pretrained", num_labels=3)  # Positive, Negative, Neutral

# Load Financial PhraseBank
dataset = load_dataset("financial_phrasebank", "sentences_50agree")  # Choose your version

# Preprocess similarly
def preprocess_fin(example):
    return tokenizer(example['sentence'], padding="max_length", truncation=True)

encoded_fin = dataset.map(preprocess_fin, batched=True)
encoded_fin.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# New training args
finetune_args = TrainingArguments(
    output_dir="./finbert_finetuned",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=finetune_args,
    train_dataset=encoded_fin['train'],
    eval_dataset=encoded_fin['test'],
)

trainer.train()